In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")

In [3]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [4]:
df['industry'] = df['industry'].fillna('NA')

# Trouver le mode (valeur la plus fréquente)
mode_industry = df['industry'].mode()[0]

print("Mode de la colonne 'industry' :", mode_industry)

# Pour vérifier la fréquence
print("\nFréquence des valeurs les plus courantes :")
print(df['industry'].value_counts().head())

Mode de la colonne 'industry' : retail

Fréquence des valeurs les plus courantes :
industry
retail        203
finance       200
other         198
healthcare    187
education     187
Name: count, dtype: int64


In [5]:


# Remplacer les valeurs manquantes
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

df[categorical] = df[categorical].fillna('NA')
df[numerical] = df[numerical].fillna(0.0)

# Sélection des variables numériques
df_num = df[numerical]

# Matrice de corrélation
corr_matrix = df_num.corr()

print("Matrice de corrélation :\n")
print(corr_matrix, "\n")

# Extraire les corrélations demandées :
pairs = {
    "interaction_count & lead_score": corr_matrix.loc['interaction_count', 'lead_score'],
    "number_of_courses_viewed & lead_score": corr_matrix.loc['number_of_courses_viewed', 'lead_score'],
    "number_of_courses_viewed & interaction_count": corr_matrix.loc['number_of_courses_viewed', 'interaction_count'],
    "annual_income & interaction_count": corr_matrix.loc['annual_income', 'interaction_count']
}

# Afficher les corrélations des paires demandées
for k, v in pairs.items():
    print(f"{k}: {v:.3f}")

# Identifier la paire avec la corrélation la plus élevée
max_pair = max(pairs, key=pairs.get)
print(f"\n✅ La plus grande corrélation est entre : {max_pair}")


Matrice de corrélation :

                          number_of_courses_viewed  annual_income  \
number_of_courses_viewed                  1.000000       0.009770   
annual_income                             0.009770       1.000000   
interaction_count                        -0.023565       0.027036   
lead_score                               -0.004879       0.015610   

                          interaction_count  lead_score  
number_of_courses_viewed          -0.023565   -0.004879  
annual_income                      0.027036    0.015610  
interaction_count                  1.000000    0.009888  
lead_score                         0.009888    1.000000   

interaction_count & lead_score: 0.010
number_of_courses_viewed & lead_score: -0.005
number_of_courses_viewed & interaction_count: -0.024
annual_income & interaction_count: 0.027

✅ La plus grande corrélation est entre : annual_income & interaction_count


In [7]:

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif

# Charger le dataset
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")

# Colonnes catégorielles et numériques
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

# Remplacement des valeurs manquantes
df[categorical] = df[categorical].fillna('NA')
df[numerical] = df[numerical].fillna(0.0)

# Séparer la cible
y = df['converted']
X = df.drop('converted', axis=1)

# Division en train / test (temporaire)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Division du train en train / val
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)
# (0.25 de 0.8 = 0.2, donc 60 / 20 / 20)

# Encoder les variables catégorielles (transformer en catégories numériques)
X_train_encoded = X_train[categorical].apply(lambda col: col.astype('category').cat.codes)

# Calculer le score d’information mutuelle
mi_scores = mutual_info_classif(X_train_encoded, y_train, discrete_features=True, random_state=42)

# Créer un DataFrame pour les résultats
mi_results = pd.Series(mi_scores, index=categorical).sort_values(ascending=False)
mi_results = mi_results.round(2)

print("Scores d'information mutuelle :\n")
print(mi_results)
print("\n✅ Variable avec le score le plus élevé :", mi_results.idxmax())


Scores d'information mutuelle :

lead_source          0.04
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

✅ Variable avec le score le plus élevé : lead_source


In [8]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chargement des données
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
df = pd.read_csv(url)

# 1. Traitement des valeurs manquantes selon les instructions
categorical_features = ['lead_source', 'industry', 'employment_status', 'location']
numerical_features = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

# Variables catégorielles → "NA"
for col in categorical_features:
    df[col] = df[col].fillna('NA')

# Variables numériques → 0.0
for col in numerical_features:
    df[col] = df[col].fillna(0.0)

# 2. Séparation des features et de la target
X = df.drop('converted', axis=1)
y = df['converted']

# 3. Split train/validation (généralement 80/20 ou 70/30)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")

# 4. Préprocessing avec One-Hot Encoding pour les variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])

# 5. Pipeline avec preprocessing et modèle Logistic Regression
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
])

# 6. Entraînement du modèle
model.fit(X_train, y_train)

# 7. Prédiction sur l'ensemble de validation
y_val_pred = model.predict(X_val)

# 8. Calcul de la précision
accuracy = accuracy_score(y_val, y_val_pred)
accuracy_rounded = round(accuracy, 2)

print(f"\n=== RÉSULTATS ===")
print(f"Précision sur l'ensemble de validation: {accuracy:.4f}")
print(f"Précision arrondie à 2 décimales: {accuracy_rounded}")

# 9. Vérification des options de réponse
print(f"\n=== OPTIONS ===")
print(f"0.64 → {'✓' if accuracy_rounded == 0.64 else '✗'}")
print(f"0.74 → {'✓' if accuracy_rounded == 0.74 else '✗'}")
print(f"0.84 → {'✓' if accuracy_rounded == 0.84 else '✗'}")
print(f"0.94 → {'✓' if accuracy_rounded == 0.94 else '✗'}")

# 10. Informations supplémentaires sur les données
print(f"\n=== INFORMATIONS SUPPLÉMENTAIRES ===")
print(f"Taille de l'ensemble d'entraînement: {len(X_train)}")
print(f"Taille de l'ensemble de validation: {len(X_val)}")
print(f"Proportion de la classe positive (converted=1): {y.mean():.2%}")

# Vérification du preprocessing
print(f"\nVariables catégorielles après one-hot encoding:")
preprocessor.fit(X_train)
X_train_transformed = preprocessor.transform(X_train)
print(f"Dimensions après one-hot encoding: {X_train_transformed.shape}")

Train set: (1169, 8)
Validation set: (293, 8)

=== RÉSULTATS ===
Précision sur l'ensemble de validation: 0.8328
Précision arrondie à 2 décimales: 0.83

=== OPTIONS ===
0.64 → ✗
0.74 → ✗
0.84 → ✗
0.94 → ✗

=== INFORMATIONS SUPPLÉMENTAIRES ===
Taille de l'ensemble d'entraînement: 1169
Taille de l'ensemble de validation: 293
Proportion de la classe positive (converted=1): 61.90%

Variables catégorielles après one-hot encoding:
Dimensions après one-hot encoding: (1169, 27)


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chargement des données
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
df = pd.read_csv(url)

# Traitement des valeurs manquantes
categorical_features = ['lead_source', 'industry', 'employment_status', 'location']
numerical_features = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

for col in categorical_features:
    df[col] = df[col].fillna('NA')
for col in numerical_features:
    df[col] = df[col].fillna(0.0)

X = df.drop('converted', axis=1)
y = df['converted']

# Split train/validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Fonction pour créer le modèle avec un ensemble de features spécifique
def train_model(features_to_use):
    # Identifier les types de features pour le sous-ensemble
    cat_subset = [f for f in categorical_features if f in features_to_use]
    num_subset = [f for f in numerical_features if f in features_to_use]
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_subset),
            ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_subset)
        ])
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
    ])
    
    model.fit(X_train[features_to_use], y_train)
    y_pred = model.predict(X_val[features_to_use])
    return accuracy_score(y_val, y_pred)

# Précision avec toutes les features
all_features = categorical_features + numerical_features
original_accuracy = train_model(all_features)
print(f"Précision avec toutes les features: {original_accuracy:.6f}")

# Test en excluant chaque feature une par une
feature_differences = {}

print("\n=== ANALYSE PAR ÉLIMINATION DE FEATURES ===")
for feature in ['industry', 'employment_status', 'lead_score']:
    features_without = [f for f in all_features if f != feature]
    accuracy_without = train_model(features_without)
    difference = original_accuracy - accuracy_without
    feature_differences[feature] = difference
    
    print(f"Sans '{feature}':")
    print(f"  Précision: {accuracy_without:.6f}")
    print(f"  Différence: {difference:.6f}")

# Trouver la feature avec la plus petite différence
min_diff_feature = min(feature_differences, key=feature_differences.get)
min_diff_value = feature_differences[min_diff_feature]

print(f"\n=== RÉSULTAT ===")
print(f"La feature avec la plus petite différence est: '{min_diff_feature}'")
print(f"Avec une différence de: {min_diff_value:.6f}")

# Affichage détaillé pour vérification
print("\n=== COMPARAISON DÉTAILLÉE ===")
for feature, diff in sorted(feature_differences.items(), key=lambda x: abs(x[1])):
    print(f"'{feature}': différence = {diff:.6f}")

# Vérification des options
print(f"\n=== VÉRIFICATION DES OPTIONS ===")
print(f"'industry' → différence = {feature_differences['industry']:.6f}")
print(f"'employment_status' → différence = {feature_differences['employment_status']:.6f}")
print(f"'lead_score' → différence = {feature_differences['lead_score']:.6f}")

Précision avec toutes les features: 0.832765

=== ANALYSE PAR ÉLIMINATION DE FEATURES ===
Sans 'industry':
  Précision: 0.819113
  Différence: 0.013652
Sans 'employment_status':
  Précision: 0.805461
  Différence: 0.027304
Sans 'lead_score':
  Précision: 0.822526
  Différence: 0.010239

=== RÉSULTAT ===
La feature avec la plus petite différence est: 'lead_score'
Avec une différence de: 0.010239

=== COMPARAISON DÉTAILLÉE ===
'lead_score': différence = 0.010239
'industry': différence = 0.013652
'employment_status': différence = 0.027304

=== VÉRIFICATION DES OPTIONS ===
'industry' → différence = 0.013652
'employment_status' → différence = 0.027304
'lead_score' → différence = 0.010239


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Chargement des données
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
df = pd.read_csv(url)

# Traitement des valeurs manquantes
categorical_features = ['lead_source', 'industry', 'employment_status', 'location']
numerical_features = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

for col in categorical_features:
    df[col] = df[col].fillna('NA')
for col in numerical_features:
    df[col] = df[col].fillna(0.0)

X = df.drop('converted', axis=1)
y = df['converted']

# Split train/validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Préprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])

# Valeurs de C à tester
C_values = [0.01, 0.1, 1, 10, 100]
results = []

print("=== RÉGRESSION LOGISTIQUE RÉGULARISÉE ===")
print("Test des différentes valeurs de C:\n")

best_accuracy = 0
best_C = None

for C in C_values:
    # Modèle avec la valeur de C actuelle
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42))
    ])
    
    # Entraînement et prédiction
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    
    # Calcul de la précision
    accuracy = accuracy_score(y_val, y_val_pred)
    accuracy_rounded = round(accuracy, 3)
    
    results.append({
        'C': C,
        'accuracy': accuracy,
        'accuracy_rounded': accuracy_rounded
    })
    
    print(f"C = {C:5}: Précision = {accuracy:.6f} → Arrondi = {accuracy_rounded}")
    
    # Mise à jour de la meilleure valeur
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_C = C
        best_accuracy_rounded = accuracy_rounded

print(f"\n=== MEILLEUR RÉSULTAT ===")
print(f"Meilleur C: {best_C}")
print(f"Meilleure précision: {best_accuracy:.6f}")
print(f"Meilleure précision arrondie: {best_accuracy_rounded}")

# Affichage des résultats pour chaque option
print(f"\n=== VÉRIFICATION DES OPTIONS ===")
for result in results:
    is_best = "✓" if result['C'] == best_C else "✗"
    print(f"C = {result['C']:5} → {result['accuracy_rounded']} {is_best}")

# Analyse supplémentaire : impact de la régularisation
print(f"\n=== ANALYSE DE LA RÉGULARISATION ===")
print("C plus petit = plus de régularisation (pénalité plus forte)")
print("C plus grand = moins de régularisation (pénalité plus faible)")

# Vérification du nombre d'itérations nécessaires
print(f"\n=== CONVERGENCE ===")
for C in C_values:
    model_test = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42))
    ])
    model_test.fit(X_train, y_train)
    n_iter = model_test.named_steps['classifier'].n_iter_[0]
    print(f"C = {C:5}: {n_iter} itérations")

=== RÉGRESSION LOGISTIQUE RÉGULARISÉE ===
Test des différentes valeurs de C:

C =  0.01: Précision = 0.815700 → Arrondi = 0.816
C =   0.1: Précision = 0.815700 → Arrondi = 0.816
C =     1: Précision = 0.832765 → Arrondi = 0.833
C =    10: Précision = 0.832765 → Arrondi = 0.833
C =   100: Précision = 0.839590 → Arrondi = 0.84

=== MEILLEUR RÉSULTAT ===
Meilleur C: 100
Meilleure précision: 0.839590
Meilleure précision arrondie: 0.84

=== VÉRIFICATION DES OPTIONS ===
C =  0.01 → 0.816 ✗
C =   0.1 → 0.816 ✗
C =     1 → 0.833 ✗
C =    10 → 0.833 ✗
C =   100 → 0.84 ✓

=== ANALYSE DE LA RÉGULARISATION ===
C plus petit = plus de régularisation (pénalité plus forte)
C plus grand = moins de régularisation (pénalité plus faible)

=== CONVERGENCE ===
C =  0.01: 4 itérations
C =   0.1: 5 itérations
C =     1: 6 itérations
C =    10: 6 itérations
C =   100: 6 itérations
